In [25]:
import pandas as pd
import time
import os

In [ ]:
# Define the industry range
sectors = range(10, 65, 5)

# Set the data directory path - hardcoded as my_outputs
DATA_DIR = "./data_processor/my_outputs"
FUNDAMENTAL_FILE = f"{DATA_DIR}/final_ratios.csv"

print(f"Using data directory: {DATA_DIR}")
print(f"Fundamental data file: {FUNDAMENTAL_FILE}")
print(f"Industry range: {list(sectors)}")

In [27]:
#gvkey is unique identifier
df_dict = {'gvkey':[], 'predicted_return':[], 'trade_date':[]}

In [ ]:
# ===== New Code: Run Stock Selection for All Sectors in the my_outputs Directory =====
start=time.time()
print("Starting to run the stock selection model for all sectors...")

for sector in sectors:
    sector_file = f"{DATA_DIR}/sector{sector}.xlsx"
    print(f"\nProcessing sector{sector}...")
    
    # Run Model Training - Using Files in the my_outputs Directory
    cmd = f"python fundamental_run_model.py -sector_name sector{sector} -tic_column gvkey -fundamental {FUNDAMENTAL_FILE} -sector {sector_file}"
    print(f"Command executed: {cmd}")
    
    os.system(cmd)
    
    # Read Prediction Results
    df = pd.read_csv(f"results/sector{sector}/df_predict_best.csv", index_col=0)
    print(f"  Reading prediction results: {df.shape[0]} dates, {df.shape[1]} stocks")
    
    for idx in df.index:
        predicted_return = df.loc[idx]
        top_q = predicted_return.quantile(0.75)
        predicted_return = predicted_return[predicted_return >= top_q]
        for gvkey in predicted_return.index:
            df_dict["gvkey"].append(gvkey)
            df_dict["predicted_return"].append(predicted_return[gvkey])
            df_dict["trade_date"].append(idx)
    
    print(f"  ✓ sector{sector} Processing completed")

end=time.time()

# ===== Existing code (commented) =====
# start=time.time()
# for sector in sectors:
#     os.system(f"python3 fundamental_run_model.py -sector_name sector{sector} -tic_column gvkey -fundamental final_ratios.csv -sector sector{sector}.xlsx ")
#     df = pd.read_csv(f"results/sector{sector}/df_predict_best.csv", index_col=0)
#     for idx in df.index:
#         predicted_return = df.loc[idx]
#         top_q = predicted_return.quantile(0.75)
#         predicted_return = predicted_return[predicted_return >= top_q]
#         for gvkey in predicted_return.index:
#             df_dict["gvkey"].append(gvkey)
#             df_dict["predicted_return"].append(predicted_return[gvkey])
#             df_dict["trade_date"].append(idx)
# end=time.time()

In [ ]:
print("Total time taken: ", (end-start)/60, ' minutes')
print(f"Processing complete! Total records: {len(df_dict['gvkey'])}")

# Existing code (commented)
# print("it took ", (end-start)/60, ' minutes')

总耗时: 0.40141645272572835 分钟
处理完成！总记录数: 0


In [6]:
df_result = pd.DataFrame(df_dict)

In [ ]:
# Save results to a CSV file - using a new filename
output_file = "stock_selected_my_outputs.csv"
df_result.to_csv(output_file, index=False)
print(f"Results saved to: {output_file}")

# Existing code (commented)
# df_result.to_csv("stock_selected.csv")

结果已保存到: stock_selected_my_outputs.csv


In [9]:
df_result.head()

,gvkey,predicted_return,trade_date
